<a href="https://colab.research.google.com/github/hemekci/RecommendationSystems/blob/main/item_based_user_based_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Memory-Based Collaborative Filtering
# Hybrid Recommender System
# User-based / Item-based recommenders


MovieLens Data https://grouplens.org/datasets/movielens/

Small: 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users. 
Last updated 9/2018.

movies.csv : movieId, title, genres, rating.csv :  userId movieId rating timeStamp

We will find first the users-based recommeded 5 top movies after that 5 more recommendations will be given by item-based recommender depends on the user's last watched movies.

# User-based Recommender
1.   Prepare the Data Set and create user-movie-rating pivottable
2.   Find the Movies Watched by "the User"
3.   Accessing Data of Other Users who watches same movies
4.   Identifying Users with the Most Similar with "the User"
5.   Calculating the Weighted Average Recommendation Score for "the User"



#  1. Prepare the Data Set and create user-movie-rating pivottable



In [2]:
import pandas as pd
import numpy as np

theUserId = 19
movie = pd.read_csv('/content/drive/MyDrive/ColabData/movie_lens_dataset/movies.csv')
rating = pd.read_csv('/content/drive/MyDrive/ColabData/movie_lens_dataset/ratings.csv')
df = movie.merge(rating, how="left", on="movieId")
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [3]:
df['userId'].min()
df['userId'].max()

610.0

In [ ]:

# first - lets drop rare movies from df
thresholdRare = 30 

rating_counts = df.groupby("title").agg({"rating": "count"})
rating_counts.head()
rare_movies = rating_counts[rating_counts["rating"] <= thresholdRare].index  # index equals to movie's title
rare_movies

Index([''71 (2014)', ''Hellboy': The Seeds of Creation (2004)',
       ''Round Midnight (1986)', ''Salem's Lot (2004)',
       ''Til There Was You (1997)', ''Tis the Season for Love (2015)',
       ''burbs, The (1989)', ''night Mother (1986)',
       '*batteries not included (1987)', '...All the Marbles (1981)',
       ...
       'Zulu (2013)', '[REC] (2007)', '[REC]² (2009)',
       '[REC]³ 3 Génesis (2012)',
       'anohana: The Flower We Saw That Day - The Movie (2013)',
       'eXistenZ (1999)', 'xXx (2002)', 'xXx: State of the Union (2005)',
       '¡Three Amigos! (1986)', 'À nous la liberté (Freedom for Us) (1931)'],
      dtype='object', name='title', length=8876)

In [ ]:

common_movies = df[~df["title"].isin(rare_movies)]  # all data by checking whether the movie is in rare_movies
common_movies.shape

common_movies["title"].nunique()
common_movies.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [ ]:
common_movies.shape
common_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57391 entries, 0 to 99762
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    57391 non-null  int64  
 1   title      57391 non-null  object 
 2   genres     57391 non-null  object 
 3   userId     57391 non-null  float64
 4   rating     57391 non-null  float64
 5   timestamp  57391 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 3.1+ MB


In [ ]:
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")

In [ ]:
user_movie_df.head()

title,(500) Days of Summer (2009),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),"40-Year-Old Virgin, The (2005)",50 First Dates (2004),8 Mile (2002),A.I. Artificial Intelligence (2001),About a Boy (2002),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Adaptation (2002),Addams Family Values (1993),"Addams Family, The (1991)","Adventures of Priscilla, Queen of the Desert, The (1994)","African Queen, The (1951)",Air Force One (1997),Airplane! (1980),Akira (1988),Aladdin (1992),Alice in Wonderland (1951),Alien (1979),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Almost Famous (2000),Amadeus (1984),"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",American Beauty (1999),American Gangster (2007),American Graffiti (1973),American History X (1998),American Pie (1999),American Pie 2 (2001),"American President, The (1995)",...,Walk the Line (2005),Wallace & Gromit: A Close Shave (1995),Wallace & Gromit: The Wrong Trousers (1993),War of the Worlds (2005),Watchmen (2009),Waterworld (1995),Wayne's World (1992),Wedding Crashers (2005),"Wedding Singer, The (1998)",West Side Story (1961),What About Bob? (1991),What Women Want (2000),What's Eating Gilbert Grape (1993),When Harry Met Sally... (1989),While You Were Sleeping (1995),Whiplash (2014),Who Framed Roger Rabbit? (1988),"Whole Nine Yards, The (2000)",Wild Things (1998),Wild Wild West (1999),William Shakespeare's Romeo + Juliet (1996),Willow (1988),Willy Wonka & the Chocolate Factory (1971),Witness (1985),"Wizard of Oz, The (1939)","Wolf of Wall Street, The (2013)","World Is Not Enough, The (1999)","Wrestler, The (2008)","X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),Yes Man (2008),You've Got Mail (1998),Young Frankenstein (1974),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,4.0,NaN,NaN,4.0,5.0,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,5.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2. Find the movies watched by "the User"

In [ ]:
user_df = user_movie_df[user_movie_df.index == theUserId]
movies_watched = user_df.columns[user_df.notna().any()].tolist()
len(movies_watched)

304

# Step 3: Accessing Data of Other Users who watches same movies

In [ ]:
movies_watched_df = user_movie_df[movies_watched]
movies_watched_df.head()

title,10 Things I Hate About You (1999),101 Dalmatians (1996),2001: A Space Odyssey (1968),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),Airplane! (1980),Aladdin (1992),Alice in Wonderland (1951),Alien (1979),Alien: Resurrection (1997),Aliens (1986),American Beauty (1999),American Pie (1999),American Psycho (2000),Analyze This (1999),Animal House (1978),Antz (1998),Arachnophobia (1990),Armageddon (1998),Army of Darkness (1993),As Good as It Gets (1997),Austin Powers: International Man of Mystery (1997),Austin Powers: The Spy Who Shagged Me (1999),Babe (1995),Back to the Future (1985),Back to the Future Part II (1989),Back to the Future Part III (1990),Basic Instinct (1992),Batman & Robin (1997),Batman (1989),Batman Forever (1995),Batman Returns (1992),Beavis and Butt-Head Do America (1996),Beetlejuice (1988),Being John Malkovich (1999),Beverly Hills Cop III (1994),Big (1988),Big Daddy (1999),...,Terminator 2: Judgment Day (1991),"Terminator, The (1984)",The Devil's Advocate (1997),Thelma & Louise (1991),There's Something About Mary (1998),"Thomas Crown Affair, The (1999)",Time Bandits (1981),Tin Cup (1996),Tommy Boy (1995),Tomorrow Never Dies (1997),Total Recall (1990),Toy Story (1995),Toy Story 2 (1999),Trading Places (1983),Tron (1982),True Lies (1994),"Truth About Cats & Dogs, The (1996)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Twister (1996),Vertigo (1958),Wag the Dog (1997),Wallace & Gromit: A Close Shave (1995),Wallace & Gromit: The Wrong Trousers (1993),Waterworld (1995),Wayne's World (1992),"Wedding Singer, The (1998)",What About Bob? (1991),When Harry Met Sally... (1989),While You Were Sleeping (1995),Who Framed Roger Rabbit? (1988),"Whole Nine Yards, The (2000)",Wild Things (1998),Wild Wild West (1999),Willow (1988),Willy Wonka & the Chocolate Factory (1971),"World Is Not Enough, The (1999)","X-Files: Fight the Future, The (1998)",X-Men (2000),You've Got Mail (1998),Young Frankenstein (1974)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,4.0,5.0,NaN,4.0,NaN,3.0,NaN,4.0,4.0,NaN,4.0,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,4.0,4.0,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,5.0,4.0,4.0,NaN,NaN,5.0,NaN,4.0,NaN,4.0,5.0,NaN,NaN,5.0,NaN,5.0
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]
user_movie_count.head()

,userId,movie_count
0,1.0,98
1,2.0,2
2,3.0,4
3,4.0,54
4,5.0,17


In [ ]:
users_same_movies = user_movie_count[user_movie_count["movie_count"] > 20]["userId"]  # min 20 movies
users_same_movies.head()
users_same_movies.count()
users_same_movies.index

Int64Index([  0,   3,   5,   6,   7,  10,  14,  15,  16,  17,
            ...
            597, 598, 600, 601, 602, 603, 604, 605, 606, 608],
           dtype='int64', length=312)

# Step 4: Identifying Users with the Most Similar with "the User"

In [ ]:
umr = movies_watched_df[movies_watched_df.index.isin(users_same_movies)]
tumr = user_df[movies_watched]
# add the user to all user-movie-rating pivot df
final_df = pd.concat([umr, tumr])

final_df.head()

title,10 Things I Hate About You (1999),101 Dalmatians (1996),2001: A Space Odyssey (1968),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),Airplane! (1980),Aladdin (1992),Alice in Wonderland (1951),Alien (1979),Alien: Resurrection (1997),Aliens (1986),American Beauty (1999),American Pie (1999),American Psycho (2000),Analyze This (1999),Animal House (1978),Antz (1998),Arachnophobia (1990),Armageddon (1998),Army of Darkness (1993),As Good as It Gets (1997),Austin Powers: International Man of Mystery (1997),Austin Powers: The Spy Who Shagged Me (1999),Babe (1995),Back to the Future (1985),Back to the Future Part II (1989),Back to the Future Part III (1990),Basic Instinct (1992),Batman & Robin (1997),Batman (1989),Batman Forever (1995),Batman Returns (1992),Beavis and Butt-Head Do America (1996),Beetlejuice (1988),Being John Malkovich (1999),Beverly Hills Cop III (1994),Big (1988),Big Daddy (1999),...,Terminator 2: Judgment Day (1991),"Terminator, The (1984)",The Devil's Advocate (1997),Thelma & Louise (1991),There's Something About Mary (1998),"Thomas Crown Affair, The (1999)",Time Bandits (1981),Tin Cup (1996),Tommy Boy (1995),Tomorrow Never Dies (1997),Total Recall (1990),Toy Story (1995),Toy Story 2 (1999),Trading Places (1983),Tron (1982),True Lies (1994),"Truth About Cats & Dogs, The (1996)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Twister (1996),Vertigo (1958),Wag the Dog (1997),Wallace & Gromit: A Close Shave (1995),Wallace & Gromit: The Wrong Trousers (1993),Waterworld (1995),Wayne's World (1992),"Wedding Singer, The (1998)",What About Bob? (1991),When Harry Met Sally... (1989),While You Were Sleeping (1995),Who Framed Roger Rabbit? (1988),"Whole Nine Yards, The (2000)",Wild Things (1998),Wild Wild West (1999),Willow (1988),Willy Wonka & the Chocolate Factory (1971),"World Is Not Enough, The (1999)","X-Files: Fight the Future, The (1998)",X-Men (2000),You've Got Mail (1998),Young Frankenstein (1974)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,4.0,5.0,NaN,4.0,NaN,3.0,NaN,4.0,4.0,NaN,4.0,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,4.0,4.0,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,5.0,4.0,4.0,NaN,NaN,5.0,NaN,4.0,NaN,4.0,5.0,NaN,NaN,5.0,NaN,5.0
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
6.0,NaN,NaN,NaN,NaN,3.0,2.0,3.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,4.0,5.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
7.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,0.5,3.5,2.0,NaN,5.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.5,5.0,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,4.5,4.5,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,3.5,NaN,NaN
8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
final_df.T

userId,1.0,4.0,6.0,7.0,8.0,11.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,24.0,27.0,28.0,31.0,32.0,33.0,34.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,50.0,51.0,56.0,57.0,58.0,59.0,62.0,63.0,64.0,66.0,68.0,...,541.0,542.0,543.0,552.0,554.0,555.0,559.0,560.0,561.0,562.0,566.0,567.0,570.0,572.0,573.0,577.0,579.0,580.0,584.0,586.0,587.0,588.0,590.0,591.0,592.0,593.0,594.0,596.0,597.0,599.0,600.0,602.0,603.0,604.0,605.0,606.0,607.0,608.0,610.0,19.0
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,3.5,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,NaN,3.0,NaN,5.0,NaN,NaN,NaN,NaN,3.0
101 Dalmatians (1996),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,1.0
2001: A Space Odyssey (1968),NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,2.0,3.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,5.0,NaN,NaN,4.0,NaN,4.5,5.0,NaN,5.0,NaN,5.0,NaN,3.5,1.0,NaN,1.5,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,1.0,NaN,5.0,NaN,3.0,NaN,3.5,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.0,NaN,5.0,4.0,NaN,5.0,NaN,NaN,5.0,NaN,3.0,4.5,3.0
"Abyss, The (1989)",4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.0,3.5,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.0,3.0,NaN,NaN,NaN,4.5,NaN,0.5,3.0,NaN,4.0,NaN,NaN,NaN,NaN,3.5,3.0,NaN,NaN,NaN,NaN,NaN,3.5,3.5,NaN,1.0,NaN,NaN,NaN,NaN,3.0,4.0,3.0
Ace Ventura: Pet Detective (1994),NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.5,2.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,2.0,4.0,NaN,4.0,3.0,NaN,3.0,5.0,NaN,1.0,NaN,3.0,5.0,3.5,NaN,2.5,...,3.0,NaN,4.0,NaN,NaN,4.0,3.0,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,3.0,3.0,3.0,NaN,4.0,1.5,NaN,NaN,NaN,2.5,3.0,2.0,NaN,2.0,NaN,NaN,NaN,3.5,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"World Is Not Enough, The (1999)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,3.5,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,5.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,2.0
"X-Files: Fight the Future, The (1998)",NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,3.0
X-Men (2000),5.0,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,4.0,4.0,3.5,3.5,NaN,NaN,3.5,NaN,NaN,NaN,5.0,NaN,4.0,NaN,NaN,2.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.5,3.5,NaN,4.5,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.5,3.5,NaN,NaN,2.5,4.0,NaN,4.5,NaN,NaN,4.5,NaN,5.0,NaN,NaN,3.0,4.0,NaN,3.0,4.5,4.0,NaN,3.5,3.0,NaN,NaN,NaN,NaN,NaN,3.0,4.0,3.5,4.0


In [ ]:
final_df.corr()
final_df.T.corr().unstack()

userId  userId
1.0     1.0       1.000000
        4.0       0.074685
        6.0      -0.448958
        7.0      -0.154145
        8.0       0.718750
                    ...   
19.0    606.0     0.314934
        607.0    -0.000504
        608.0     0.185304
        610.0     0.446994
        19.0      1.000000
Length: 97969, dtype: float64

In [ ]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df

userId  userId
215.0   169.0    -1.000000
346.0   75.0     -1.000000
325.0   46.0     -1.000000
476.0   337.0    -1.000000
586.0   488.0    -0.994135
                    ...   
79.0    587.0     1.000000
113.0   141.0     1.000000
610.0   610.0     1.000000
276.0   586.0     1.000000
4.0     119.0          NaN
Length: 37488, dtype: float64

In [ ]:
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
corr_df.head()

,user_id_1,user_id_2,0
0,215.0,169.0,-1.000000
1,346.0,75.0,-1.000000
2,325.0,46.0,-1.000000
3,476.0,337.0,-1.000000
4,586.0,488.0,-0.994135


In [ ]:
corr_df.columns = ['user_id_1', 'user_id_2', 'corr']

In [ ]:
corr_df[corr_df["user_id_1"] == theUserId]


,user_id_1,user_id_2,corr
2281,19.0,89.0,-0.365725
4281,19.0,321.0,-0.223572
5088,19.0,420.0,-0.183533
5218,19.0,541.0,-0.177405
5240,19.0,374.0,-0.176555
...,...,...,...
31141,19.0,96.0,0.448711
31719,19.0,18.0,0.464967
33087,19.0,44.0,0.511159
34517,19.0,268.0,0.575345


In [ ]:
top_users = corr_df[(corr_df["user_id_1"] == theUserId) & (corr_df["corr"] >= 0.40)][["user_id_2", "corr"]].reset_index(drop=True)
top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)
len(top_users)


12

In [ ]:
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
top_users_ratings = top_users_ratings[top_users_ratings["userId"] != theUserId]
top_users_ratings["userId"].unique()
top_users_ratings.head()


,userId,corr,movieId,rating
0,382.0,0.734934,1,4.5
1,382.0,0.734934,50,5.0
2,382.0,0.734934,150,4.0
3,382.0,0.734934,186,3.0
4,382.0,0.734934,208,3.5


# 5.   Calculating the Weighted Average Recommendation Score for "the User"


In [ ]:

top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings['weighted_rating'].head()
top_users_ratings.head()
top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})

recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index()
recommendation_df.head()


,movieId,weighted_rating
0,1,1.978634
1,2,1.311497
2,3,1.585468
3,5,0.818730
4,6,1.620939


In [ ]:
recommendation_df[["weighted_rating"]].describe()

,weighted_rating
count,3539.000000
mean,1.500788
std,0.428643
min,0.213955
25%,1.228095
50%,1.619976
75%,1.738326
max,3.307205


In [ ]:
# weighted_rating greater than 2.5 -  
recommendation_df[recommendation_df["weighted_rating"] > 2.5]
len(recommendation_df[recommendation_df["weighted_rating"] > 2.5])
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 2.5].sort_values("weighted_rating", ascending=False)[0:5]
movies_to_be_recommend.head()

,movieId,weighted_rating
2992,72737,3.307205
2824,56152,3.307205
3024,79185,2.939738
2869,59501,2.939738
2847,58105,2.939738


In [ ]:
movies_to_be_recommend.merge(movie[["movieId", "title"]])["title"]


0                   Princess and the Frog, The (2009)
1                                    Enchanted (2007)
2                               Knight and Day (2010)
3    Chronicles of Narnia: Prince Caspian, The (2008)
4                   Spiderwick Chronicles, The (2008)
Name: title, dtype: object

# ITEM BASED

In [ ]:
movie_id = rating[(rating["userId"] == theUserId) & (rating["rating"] == 5.0)].sort_values(by="timestamp", ascending=False)["movieId"][0:1].values[0]


def item_based_recommender(movie_name, user_movie_df):
  movie = user_movie_df[movie_name]
  return user_movie_df.corrwith(movie).sort_values(ascending=False).head(10)

movie_id

904

In [ ]:
movies_from_item_based = item_based_recommender(movie[movie["movieId"] == movie_id]["title"].values[0], user_movie_df)


In [ ]:
movies_from_item_based[1:10].index


Index(['Intouchables (2011)', 'Devil Wears Prada, The (2006)', 'Rudy (1993)',
       'Philadelphia (1993)', 'Four Weddings and a Funeral (1994)',
       'Big Hero 6 (2014)', 'Super Size Me (2004)', '12 Angry Men (1957)',
       'Grumpier Old Men (1995)'],
      dtype='object', name='title')

In [ ]:
item_based_recommender("Devil Wears Prada, The (2006)", user_movie_df)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


title
Executive Decision (1996)                         1.0
Go (1999)                                         1.0
Heavy Metal (1981)                                1.0
Wag the Dog (1997)                                1.0
Insomnia (2002)                                   1.0
Madness of King George, The (1994)                1.0
Devil Wears Prada, The (2006)                     1.0
Species (1995)                                    1.0
Sgt. Bilko (1996)                                 1.0
Mystery Science Theater 3000: The Movie (1996)    1.0
dtype: float64